# TP2: Propiedades en venta

## Grupo #06: Adelsflügel Valentina, Emanuel Tomás, Javes Sofía, Frenkel Gastón, Zacarías Víctor

# Procesamiento del lenguaje natural (PLN)

Se conoce como lenguaje natural a aquel destinado a a la comunicación entre personas, tal como lo es el Inglés, o el Castellano. 

Cuando se habla de procesamiento del lenguaje natural o PLN, se hace referencia a un conjunto de técnicas o algoritmos que permiten procesar subconjuntos de dicho lenguaje (no se analiza todo el lenguaje en si, sino frases, palabras u oraciones) con el objetivo de analizarlos en busca de información que resulte relevante a algún tipo de problema. 

Como se mencionó, PLN tiene como objetivo procesar subconjuntos del lenguaje natural con el objetivo de buscar información relevante en ellos. Por información relevante se hace referencia a si un determinado texto pertenece a una clase (por ejemplo, si una crítica es negativa o positiva).

En lo que respecta al trabajo, se dispone de un conjunto de datos con descripciones en Castellano de propiedades inmobiliarias. Al usar PLN sobre dicho conjunto, se buscará analizar las descripciones en busca de características relevantes sobre las propiedades que permitan clasificarlas según qué característica posean. Posteriormente se generaran nuevas columnas sobre el dataset original con dichas características.

Se importan las bibliotecas a utilizar:

In [1]:
import os
import pandas as pd
import numpy as np
import nltk as nltk
nltk.download('sentiwordnet')
nltk.download('wordnet')
from nltk.corpus import sentiwordnet as swn
from nltk import word_tokenize
from nltk.collocations import *
from nltk.stem.wordnet import WordNetLemmatizer
import re

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Se importan los datasets a utilizar:

In [2]:
ds_properati_train = pd.read_csv("datasets/dataset_properati_train_id.csv")
ds_properati_test = pd.read_csv("datasets/dataset_properati_test_id.csv")
ds_properati_test.drop(columns="Unnamed: 0", inplace=True)
ds_properati_train.drop(columns="Unnamed: 0", inplace=True)

descripcion_propiedades_ds = pd.read_csv("datasets/dataset_properati_descrip.csv")

Al ser la primera vez que se trabaja con el dataset 'descripcion_propiedades_ds' se realiza un análisis del mismo, de forma de depurarlo y dejarlo con la información mas importante para los objetivos del trabajo.

## Análisis exploratorio descripcion_propiedades_ds

In [3]:
descripcion_propiedades_ds.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...


In [4]:
descripcion_propiedades_ds.shape

(460154, 2)

In [5]:
descripcion_propiedades_ds.dtypes

id                      object
property_description    object
dtype: object

Se tiene un dataset de 460154x2 (Filas x columnas). Ambas columnas son de tipo object (string a fines prácticos).

-Análisis de valores faltantes:

In [6]:
descripcion_propiedades_ds.isna().sum()

id                       0
property_description    23
dtype: int64

Se observan 23 datos nulos, o valores faltantes,  en la columna property_description.  A continuación se realiza un análisis sobre cual es el porcentaje de datos nulos para tomar una decisión sobre qué hacer con ellos.

In [7]:
descripciones_nulas = descripcion_propiedades_ds['property_description'].isna().sum()
descripciones_totales = descripcion_propiedades_ds['property_description'].size

porcentaje_de_datos_nulos = (descripciones_nulas*100)/descripciones_totales

print("El porcentaje de datos nulos es:",porcentaje_de_datos_nulos,'%')

El porcentaje de datos nulos es: 0.004998326647165948 %


Como se puede observar el  porcentaje de datos nulos es muy bajo, por lo tanto los mismos no aportan ningún tipo de información relevante a los objetivos del trabajo, resultando innecesario hacer algún análisis adicional sobre ellos. 

Teniendo en cuenta lo observado anteriormente los datos nulos son eliminados del dataset.

In [8]:
descripcion_propiedades_ds.dropna(inplace=True)
descripcion_propiedades_ds.reset_index(inplace=True, drop=True)

In [9]:
descripcion_propiedades_ds.isna().sum()

id                      0
property_description    0
dtype: int64

A continuación haremos un matcheo entre las descripciones y las propiedades utilizando el id para quedarnos sólo con las que cumplen las condiciones de las propiedades que nos interesan (venta, CABA, dólares, etc).

Separamos las descripciones en test y train para poder hacer más rápido el matcheo con los respectivos datasets de train y test del tp1.

In [10]:
descripciones_train = descripcion_propiedades_ds[descripcion_propiedades_ds['id'].isin(ds_properati_train["id"])]
descripciones_test = descripcion_propiedades_ds[descripcion_propiedades_ds['id'].isin(ds_properati_test["id"])]

In [11]:
#juntamos todas las descripciones
frames = [descripciones_test, descripciones_train]
ds_descripciones = pd.concat(frames)
ds_descripciones.reset_index(drop=True, inplace=True)
ds_descripciones

,id,property_description
0,uH5H+RfQM87Qtrb13+T4hA==,"Venta PH 4 ambientes Paternal<br><br>Cálido, ..."
1,3pM/roNdUDuYB1d+EgIPyg==,Hermoso Monoambiente amplio: 48m2 (46 cubierto...
2,TFL4KKRhKUuOS9QiO4VM+g==,EXCELENTE MONOAMBIENTE <br>- MUY LUMINOSO - GR...
3,vPqbNhqDdABxvRf4mpaxow==,Monoambiente en venta super luminoso en 1 er p...
4,jMccznz3OHj2IMAxxEPCBg==,Luminoso departamento interno de dos ambientes...
...,...,...
49580,Mb3HZJicguJBZKfIbTtyug==,Corredor Responsable: Jorge Salafia - CUCICBA ...
49581,eFo0Jti9Eh4xZ31e+mIp5A==,Departamento en VENTA 2 Amb. c/Patio<br><br>Se...
49582,b3ZWAXpydmLTSikv5Co8vQ==,A menos de 200 mts de Av. Jonte<br><br>PH 3 AM...
49583,TOFXFoob32z5R9t1IH5/eQ==,Hermosa Casa en Villa del Parque de mas de 500...


In [12]:
#exportamos a un txt
np.savetxt(r'descripcionestp1.txt', ds_descripciones.property_description, fmt='%s')
#lo abrimos
descripciones_txt = open("descripcionestp1.txt", "r")

In [13]:
descripciones_train

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
10,+9uGr8tPEUkKnpjOef+gUw==,SE VENDE EXCELENTE PH de 2 CÓMODOS AMBIENTES M...
12,tto06YVR54II+aQABaJ6ug==,PH de 2 ambientes en Mataderos. Desarrollado e...
...,...,...
460049,Mb3HZJicguJBZKfIbTtyug==,Corredor Responsable: Jorge Salafia - CUCICBA ...
460051,eFo0Jti9Eh4xZ31e+mIp5A==,Departamento en VENTA 2 Amb. c/Patio<br><br>Se...
460052,b3ZWAXpydmLTSikv5Co8vQ==,A menos de 200 mts de Av. Jonte<br><br>PH 3 AM...
460055,TOFXFoob32z5R9t1IH5/eQ==,Hermosa Casa en Villa del Parque de mas de 500...


In [14]:
descripciones_test

,id,property_description
11,uH5H+RfQM87Qtrb13+T4hA==,"Venta PH 4 ambientes Paternal<br><br>Cálido, ..."
339,3pM/roNdUDuYB1d+EgIPyg==,Hermoso Monoambiente amplio: 48m2 (46 cubierto...
341,TFL4KKRhKUuOS9QiO4VM+g==,EXCELENTE MONOAMBIENTE <br>- MUY LUMINOSO - GR...
348,vPqbNhqDdABxvRf4mpaxow==,Monoambiente en venta super luminoso en 1 er p...
380,jMccznz3OHj2IMAxxEPCBg==,Luminoso departamento interno de dos ambientes...
...,...,...
459873,lmQCwvl2evBh5qnlqpVRKg==,VENTA PH 3 AMBIENTES CON BALCÓN CORRIDO EN VIL...
460047,JhkKNOeIIUm5iD0v7ilDDw==,Corredor Responsable: Ariel Champanier / Andre...
460050,PspUXP71+JWo7BtLaGP/oQ==,Se trata de un PH de 4 ambientes con patio y g...
460056,qJL7q4skB3DuPwnj0Pg0Gw==,VENTA DE PH DE DOS AMBIENTES EN PARQUE AVELLAN...


In [15]:
ds_properati_test.shape

(9866, 14)

In [16]:
ds_properati_train.shape

(39719, 14)

Vemos que todas las propiedades tienen su descripcion.

In [17]:
def match_descripcion(ds_properati, rslt):    
    for index, row in rslt.iterrows():
        id_ = row["id"]
        descripcion = row.property_description
        registro = ds_properati.loc[ds_properati["id"] == id_]
        ds_properati.at[registro.index,'descripcion'] = descripcion

In [18]:
match_descripcion(ds_properati_test, descripciones_test)
match_descripcion(ds_properati_train, descripciones_train)

In [19]:
ds_properati_train.head()

,id,created_on,latitud,longitud,provincia,barrio,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,descripcion
0,Tl/0P3OQvAJx+s3S6k1YTQ==,2021-03-10,-34.611523,-58.410394,Capital Federal,BALVANERA,Venta,Departamento,1.0,1.0,49.0,45.0,79000.0,USD,EN CONSTRUCCION a entregar en Septiembre 202...
1,OwVESiGxXI0GQgEX5TuguA==,2021-04-16,-34.621782,-58.424452,Capital Federal,ALMAGRO,Venta,Departamento,1.0,1.0,44.0,40.0,99000.0,USD,Corredor Responsable: Emilia Silvia Pepe - CUC...
2,LikTulyE2kKBjdKCrPTvOg==,2021-08-28,-34.592425,-58.515042,Capital Federal,VILLA DEVOTO,Venta,PH,3.0,2.0,76.0,66.0,160000.0,USD,Corredor Responsable: Mario Gabriel Gerez - CM...
3,yRW3du6JKpSpTVstJJC6Yg==,2021-11-29,-34.607836,-58.446965,Capital Federal,CABALLITO,Venta,Departamento,2.0,1.0,56.0,50.0,153000.0,USD,Departamento en duplex de 56 m2. Cochera fija....
4,nd95bu4KbYrpHdurPqv3/A==,2021-12-17,-34.561456,-58.467606,Capital Federal,BELGRANO,Venta,Departamento,2.0,1.0,55.0,45.0,159000.0,USD,Estratégicamente ubicado en la uni...


In [20]:
ds_properati_test.head()

,id,created_on,latitud,longitud,provincia,barrio,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,descripcion
0,Dabq6WG4OfnpGJafX4AH4w==,2021-11-09,-34.552834,-58.491372,Capital Federal,SAAVEDRA,Venta,PH,5.0,3.0,158.000000,115.0,220000.0,USD,"Se vende en el tradicional barrio de ""SAAVEDRA..."
1,+b4ao+uuJ47NJGpJMuLP2w==,2021-03-23,-34.580101,-58.412473,Capital Federal,PALERMO,Venta,PH,2.0,1.0,35.000000,32.0,90000.0,USD,Excelente PH en la mejor zona de Palermo.<br><...
2,sjZQWQmA3Jgw7HtWWU0bpw==,2021-10-05,-34.598510,-58.411857,Capital Federal,ABASTO,Venta,Departamento,2.0,1.0,43.000000,41.0,85000.0,USD,Corredor Responsable: JUAN FRANCISCO CARATTINO...
3,/uazrulgE/zCLdo6x1Xvhw==,2021-05-15,-34.567515,-58.458584,Capital Federal,BELGRANO,Venta,Departamento,2.0,1.0,111.325111,44.0,144000.0,USD,VENTA DEPARTAMENTO<br><br>Impecable departamen...
4,wirKqwH+3UYdAFVO5NVp4A==,2021-09-02,-34.588866,-58.416343,Capital Federal,PALERMO,Venta,Departamento,2.0,1.0,39.000000,36.0,105000.0,USD,"Departamento de 2 ambientes al frente, apto p..."


## Búsqueda de aspectos de propiedades: El método de Bing Liu y Minqing Hu

Este método realiza la detección de aspectos basándose en frecuencia y reglas.  Por frecuencia entendemos a aquellas palabras/frases (que no sean stopwords) que se repiten reiteradamente, y por reglas, por ejemplo, "ocurre después de una palabra que indica sentimientos".

Como en las descripciones hay una gran repetición de artículos importamos la herramienta para el análisis de lenguaje natural y descargamos la biblioteca con stopwords, que serán las palabras que queremos que se ignoren al procesar las descripciones. 

En primer lugar analizaremos las descripciones de cada propiedad seleccionando las palabras con mayor frecuencia.

In [21]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwordsSp = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Vimos que una palabra que se repite mucho es la etiqueta html \<br> que no nos aporta nada de informacion, por lo que queremos ignorarla.

In [22]:
stopwordsSp.add("br")

Verificamos que efectivamente se haya agregado 'br' al conjunto.

In [23]:
stopwordsSp

{'a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'br',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fu

Construimos entonces un vocabulario, es decir un conjunto de palabras en donde cada palabra aparece una sola vez.

Importamos y construimos el CountVectorizer para las descripciones, seteamos 30 palabras máximo ya que sino el programa demora demasiado y como stopwords ponemos el conjunto ya seteado anteriormente.

Mostramos las palabras más frecuentes con su frecuencia por descripción.

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=30, stop_words=stopwordsSp)
X = vectorizer.fit_transform(descripciones_txt)
palabras_mas_frecuentes = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out())
palabras_mas_frecuentes

,30,ambientes,amplio,av,balcón,baño,casa,cocina,comedor,completo,...,living,m2,medidas,piso,pisos,placard,propiedad,responsable,terraza,valor
0,0,2,1,0,0,1,0,1,1,0,...,2,2,1,0,1,0,2,0,2,0
1,0,0,2,1,1,1,0,1,1,1,...,1,0,1,1,1,0,1,0,0,0
2,0,0,1,1,0,2,0,2,0,1,...,0,0,0,2,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0,1,0,1,0,1,0,1,1,1,...,1,0,1,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75433,1,0,0,0,1,1,1,1,0,1,...,0,0,0,0,0,1,1,1,0,1
75434,0,0,0,2,0,1,0,1,0,1,...,1,0,1,0,0,0,2,0,0,1
75435,0,1,0,1,1,3,0,3,3,1,...,2,0,2,1,0,0,1,0,2,1
75436,0,0,0,0,0,3,2,1,1,1,...,1,0,0,1,1,0,0,0,2,0


Ordenamos de mayor a menor frecuencia

In [25]:
columnas = palabras_mas_frecuentes.columns
frec = {}
for col in columnas:
    frec[col] = palabras_mas_frecuentes[col].sum()

sorted_words = sorted(frec, reverse=True)
sorted_words

['valor',
 'terraza',
 'responsable',
 'propiedad',
 'placard',
 'pisos',
 'piso',
 'medidas',
 'm2',
 'living',
 'inmueble',
 'frente',
 'expensas',
 'excelente',
 'edificio',
 'dormitorio',
 'departamento',
 'cuenta',
 'cuadras',
 'corredor',
 'completo',
 'comedor',
 'cocina',
 'casa',
 'baño',
 'balcón',
 'av',
 'amplio',
 'ambientes',
 '30']

Podemos ver que entre las palabras más frecuentes que no hayamos tenido en cuenta todavía y que sean interesantes, se encuentran: pisos, expensas, balcón, frente, avenida (av). Estos son los potenciales aspectos que vamos a agregar como nuevas columnas al dataset. Para eso debemos analizar las descripciones para ver qué valores toman esos aspectos.

Como paso siguiente vamos a separar las descripciones en oraciones y quedarnos con aquellas que contengan los aspectos de interés.

In [26]:
#guardamos todas las descripciones en un vector
texto = []
with open('descripcionestp1.txt') as descripciones_txt:
    for line in descripciones_txt:
        texto.append(line)  
texto

['Venta PH 4 ambientes Paternal<br><br>Cálido,  divino y amplio PH 4 ambientes en Paternal, en cuadra agradable y silenciosa!<br><br>PH ampliado.  El mismo cuenta con una superficie cubierta aproximada de 89 m2  y una terraza de 38 m2. <br>Pisos de pinotea y flotantes en excelente estado!<br><br>Se encuentra ubicado al fondo de lote de PH de dos unidades.<br>Se ingresa a la propiedad por pasillo en Planta Baja. Nos encontramos con un patio interno, hermoso living, living comedor, cocina independiente  y un baño.<br>Luego por escaleras se presentan dos dormitorios y terraza acondicionada  para su uso.<br>Sin Expensas!<br><br>La propiedad se encuentra lista para mudarse!<br>Aguardamos tu consulta para poder visitarla!! <br><br>Transportes Cercanos<br>Colectivos: 24, 57, 63, 105, 109, 113, 133, 135, 146<br><br>AVISO LEGAL: Las descripciones arquitectónicas y funcionales, valores de expensas, impuestos y servicios, fotos y medidas de este inmueble son aproximados. Los datos fueron proporci

In [27]:
def split_string(string, delimiter):
    new_string = ""
    for i in range(len(string)):
        if string[i] == delimiter:
            new_string += string[i]
            new_string += "\n"
        else:            
            new_string += string[i] 
    return new_string   

In [28]:
#separamos las oraciones
delimiter = "."
descripciones = []
for i in texto:
    descripciones.append(split_string(i, delimiter))
descripciones

['Venta PH 4 ambientes Paternal<br><br>Cálido,  divino y amplio PH 4 ambientes en Paternal, en cuadra agradable y silenciosa!<br><br>PH ampliado.\n  El mismo cuenta con una superficie cubierta aproximada de 89 m2  y una terraza de 38 m2.\n <br>Pisos de pinotea y flotantes en excelente estado!<br><br>Se encuentra ubicado al fondo de lote de PH de dos unidades.\n<br>Se ingresa a la propiedad por pasillo en Planta Baja.\n Nos encontramos con un patio interno, hermoso living, living comedor, cocina independiente  y un baño.\n<br>Luego por escaleras se presentan dos dormitorios y terraza acondicionada  para su uso.\n<br>Sin Expensas!<br><br>La propiedad se encuentra lista para mudarse!<br>Aguardamos tu consulta para poder visitarla!! <br><br>Transportes Cercanos<br>Colectivos: 24, 57, 63, 105, 109, 113, 133, 135, 146<br><br>AVISO LEGAL: Las descripciones arquitectónicas y funcionales, valores de expensas, impuestos y servicios, fotos y medidas de este inmueble son aproximados.\n Los datos f

Ahora sí, exportamos a modo de documentación un txt con todas las oraciones que contienen las descripciones a ser utilizadas.

In [29]:
with open(r'oraciones_de_descripciones.txt', 'w') as fp:
    for item in descripciones:
        fp.write("%s" % item)

In [30]:
#en una lista guardamos cada oración
oraciones = []
with open(r'oraciones_de_descripciones.txt', 'r') as fp:
    for line in fp:
        oraciones.append(line)
oraciones

['Venta PH 4 ambientes Paternal<br><br>Cálido,  divino y amplio PH 4 ambientes en Paternal, en cuadra agradable y silenciosa!<br><br>PH ampliado.\n',
 '  El mismo cuenta con una superficie cubierta aproximada de 89 m2  y una terraza de 38 m2.\n',
 ' <br>Pisos de pinotea y flotantes en excelente estado!<br><br>Se encuentra ubicado al fondo de lote de PH de dos unidades.\n',
 '<br>Se ingresa a la propiedad por pasillo en Planta Baja.\n',
 ' Nos encontramos con un patio interno, hermoso living, living comedor, cocina independiente  y un baño.\n',
 '<br>Luego por escaleras se presentan dos dormitorios y terraza acondicionada  para su uso.\n',
 '<br>Sin Expensas!<br><br>La propiedad se encuentra lista para mudarse!<br>Aguardamos tu consulta para poder visitarla!! <br><br>Transportes Cercanos<br>Colectivos: 24, 57, 63, 105, 109, 113, 133, 135, 146<br><br>AVISO LEGAL: Las descripciones arquitectónicas y funcionales, valores de expensas, impuestos y servicios, fotos y medidas de este inmueble 

Vamos a quedarnos con las oraciones que tengan alguno de los aspectos antes seleccionados.


In [31]:
descripciones_aspectos_red = []
for i in oraciones:
    if('av'in i or 'frente'in i or 'balcón'in i or 'pisos'in i or 'expensas'in i):
        descripciones_aspectos_red.append(i)
print(len(descripciones_aspectos_red))
descripciones_aspectos_red

116488


['<br>Sin Expensas!<br><br>La propiedad se encuentra lista para mudarse!<br>Aguardamos tu consulta para poder visitarla!! <br><br>Transportes Cercanos<br>Colectivos: 24, 57, 63, 105, 109, 113, 133, 135, 146<br><br>AVISO LEGAL: Las descripciones arquitectónicas y funcionales, valores de expensas, impuestos y servicios, fotos y medidas de este inmueble son aproximados.\n',
 ' Los datos fueron proporcionados por el propietario y pueden no estar actualizados a la hora de la visualización de este aviso por lo cual pueden arrojar inexactitudes y discordancias con las que surgen de los las facturas, títulos y planos legales del inmueble.\n',
 '<br>Lavaderito independiente compartimentado.\n',
 '<br>Edificio de 5 pisos, con 8 unidades por piso y 2 ascensores.\n',
 ' Precio modificable sin previo aviso.\n',
 ' Último valor informado expensas Marzo $ 1600.\n',
 ' Living comedor y amplia cocina con lavadero incorporado.\n',
 '<br><br>Comercializa Pessah Propiedades CUCICBA 5678/CMCPSI 6686<br><br

Buscamos los aspectos de las palabras mas frecuentes, quedándonos así con las palabras anterior y posterior inmediatas a la aparición de la palabra con expresiones regulares (regex).

In [32]:
import re

Como regex solo funciona con string debemos convertir la lista de aspectos

In [33]:
def lista_a_string(string):

    str1 = ""
 
    for element in string:
        str1 += element
 
    return str1

Verificamos que efectivamente se haya convertido

In [34]:
descripciones_aspectos_red_s = lista_a_string(descripciones_aspectos_red)
type(descripciones_aspectos_red_s)

str

Usamos una búsqueda con regex que nos seleccione la palabra anterior y la posterior a la palabra de nuestro interés.

In [35]:
def busqueda_de_aspectos(string, palabra, pattern):

    aspecto_palabra = ""   

    aspecto_palabra = re.findall(pattern, string, re.IGNORECASE)

    return aspecto_palabra

Buscamos los aspectos posibles de las palabras mas frecuentes ya mencionadas que podrían ser candidatas a nuevas columnas

In [36]:
pattern_balcón = r"(\w+)\s(balcón)\s(\w+)"
balcon = busqueda_de_aspectos(descripciones_aspectos_red_s, 'balcón', pattern_balcón)
balcon

[('Con', 'balcón', 'al'),
 ('con', 'balcón', 'francés'),
 ('a', 'balcón', 'al'),
 ('con', 'balcón', 'francés'),
 ('un', 'balcón', 'de'),
 ('o', 'balcón', 'terraza'),
 ('gran', 'balcón', 'corrido'),
 ('con', 'balcón', 'corrido'),
 ('y', 'balcón', 'Corredor'),
 ('con', 'balcón', 'francés'),
 ('con', 'balcón', 'francés'),
 ('con', 'balcón', 'corrido'),
 ('con', 'balcón', 'corrido'),
 ('a', 'balcón', 'con'),
 ('con', 'balcón', 'al'),
 ('tienen', 'balcón', 'propio'),
 ('mas', 'balcón', 'de'),
 ('mas', 'balcón', 'de'),
 ('amplio', 'balcón', 'super'),
 ('amplio', 'balcón', 'corrido'),
 ('pequeño', 'balcón', 'con'),
 ('con', 'balcón', 'al'),
 ('y', 'balcón', 'al'),
 ('Cómodo', 'Balcón', 'al'),
 ('espacio', 'balcón', 'pequeño'),
 ('al', 'balcón', 'con'),
 ('a', 'balcón', 'al'),
 ('al', 'balcón', 'con'),
 ('un', 'balcón', 'aterrazado'),
 ('con', 'balcón', 'al'),
 ('al', 'balcón', 'a'),
 ('a', 'balcón', 'corrido'),
 ('a', 'balcón', 'aterrazado'),
 ('a', 'balcón', 'aterrazado'),
 ('al', 'balcón', 

En casos como el siguiente, notamos que si utilizamos exclusivamente la palabra anterior y posterior inmediatas la información quedaba incompleta por lo que decidimos agregar una segunda palabra para que esto no sucediera:

In [37]:
pattern_avenida = r"(\w+)\s(av)\s(\w+)\s(\w+)"
avenida = busqueda_de_aspectos(descripciones_aspectos_red_s, 'av', pattern_avenida)
avenida

[('la', 'Av', 'Luis', 'María'),
 ('sobre', 'av', 'Córdoba', 'en'),
 ('de', 'Av', 'Santa', 'Fe'),
 ('sobre', 'Av', 'Corrientes', 'al'),
 ('de', 'Av', 'Dorrego', 'y'),
 ('LA', 'AV', 'AVELLANEDAESTACION', 'DE'),
 ('de', 'Av', 'Santa', 'Fé'),
 ('de', 'Av', 'Juan', 'B'),
 ('la', 'Av', 'Juan', 'B'),
 ('la', 'Av', 'Gaona', 'y'),
 ('en', 'Av', 'Balbín', 'al'),
 ('de', 'Av', 'Crisólogo', 'Larralde'),
 ('a', 'Av', 'Rivadavia', 'y'),
 ('en', 'Av', 'Córdoba', 'yJorge'),
 ('de', 'Av', 'Alvarez', 'Thomas'),
 ('la', 'Av', 'Rivadavia', 'y'),
 ('de', 'Av', 'Rivadavia', 'y'),
 ('la', 'Av', 'Scalabrini', 'Ortiz'),
 ('de', 'Av', 'Diaz', 'Velez'),
 ('de', 'Av', 'Dorrego', 'y'),
 ('de', 'Av', 'Santa', 'Fe'),
 ('la', 'Av', '9', 'de'),
 ('calle', 'Av', 'Diaz', 'Velez'),
 ('sobre', 'Av', 'Cabildo', 'esq'),
 ('de', 'Av', 'Gral', 'Paz'),
 ('a', 'Av', 'Belgrano', 'y'),
 ('de', 'Av', 'Constituyentes', 'y'),
 ('la', 'Av', 'Saenz', 'con'),
 ('la', 'Av', 'Almancio', 'Alcorta'),
 ('la', 'av', 'Perito', 'MorenoEn'),
 (

Podemos ver que todas las descripciones que contienen el aspecto de "avenida" es porque se encuentran sobre una o estan a unas cuadras "de" ellas. Identificamos nombres de avenidas que salieron cortadas luego de aplicarles la regex como Juan B Justo y 9 de Julio.

In [38]:
pattern_frente = r"(\w+)\s(frente)\s(\w+)\s(\w+)"
frente = busqueda_de_aspectos(descripciones_aspectos_red_s, 'frente', pattern_frente)
frente

[('Al', 'Frente', 'con', 'Balcón'),
 ('al', 'frente', 'de', '50'),
 ('al', 'frente', 'sobre', 'la'),
 ('con', 'frente', 'de', 'placardsPisos'),
 ('al', 'frente', 'con', 'balcón'),
 ('Al', 'frente', 'con', 'balcon'),
 ('al', 'frente', 'con', 'balcon'),
 ('con', 'frente', 'vidriado', 'y'),
 ('Al', 'frente', 'con', 'entrada'),
 ('al', 'frente', 'a', 'la'),
 ('al', 'frente', 'con', 'cerramiento'),
 ('al', 'frente', 'con', 'baño'),
 ('de', 'frente', 'por', '16'),
 ('el', 'frente', 'de', 'un'),
 ('contra', 'frente', 'posee', 'un'),
 ('al', 'frente', 'con', 'doble'),
 ('al', 'frente', 'con', 'balcn'),
 ('El', 'frente', 'se', 'conecta'),
 ('al', 'frente', 'en', 'planta'),
 ('al', 'frente', 'y', 'con'),
 ('al', 'frente', 'a', 'la'),
 ('al', 'frente', 'con', 'luz'),
 ('al', 'frente', 'con', 'acceso'),
 ('contra', 'frente', 'con', 'parrilla'),
 ('con', 'frente', 'a', 'Av'),
 ('con', 'frente', 'a', 'la'),
 ('con', 'frente', 'de', 'placardAl'),
 ('al', 'frente', 'y', 'al'),
 ('conservaran', 'frente

In [39]:
pattern_pisos = r"(\w+)\s(piso|pisos)\s(\w+)\s(\w+)"
pisos = busqueda_de_aspectos(descripciones_aspectos_red_s, 'pisos', pattern_pisos)
pisos

[('por', 'piso', 'y', '2'),
 ('9', 'pisos', 'y', '4'),
 ('con', 'pisos', 'de', 'parquet'),
 ('por', 'piso', 'y', 'dos'),
 ('y', 'pisos', 'de', 'parquet'),
 ('19', 'pisos', 'con', 'triple'),
 ('19', 'pisos', 'con', 'triple'),
 ('2', 'pisos', 'y', 'cuenta'),
 ('primer', 'piso', 'por', 'escalera'),
 ('Primer', 'piso', 'al', 'contrafrente'),
 ('tercer', 'piso', 'por', 'ascensor'),
 ('3º', 'piso', 'por', 'Ascensor'),
 ('con', 'pisos', 'de', 'madera'),
 ('13', 'pisos', 'de', 'diseño'),
 ('en', 'pisos', '12', 'y'),
 ('en', 'pisos', '12', 'y'),
 ('es', 'piso', '6', 'al'),
 ('Los', 'pisos', 'son', 'originales'),
 ('tienen', 'pisos', 'de', 'porcelanato'),
 ('los', 'pisos', 'son', 'de'),
 ('Segundo', 'piso', 'por', 'escalera'),
 ('Primer', 'piso', 'por', 'escaleraAmplia'),
 ('con', 'piso', 'de', 'parquet'),
 ('con', 'pisos', 'de', 'madera'),
 ('primer', 'piso', 'por', 'escalera'),
 ('los', 'pisos', 'y', 'revestidas'),
 ('en', 'pisos', 'como', 'en'),
 ('los', 'pisos', 'y', 'revestidas'),
 ('con', 

A simple vista podemos observar que abundan los pisos de madera, parquet, pinotea, flotantes, cerámica o porcelanato. También podemos ver como también se puede interpretar por la cantidad de pisos del edificio o de la propiedad.

In [40]:
pattern_expensas = r"(\w+)\s(expensas)\s(\w+)\s(\w+)"
expensas = busqueda_de_aspectos(descripciones_aspectos_red_s, 'expensas', pattern_expensas)
expensas

[('y', 'expensas', 'aquí', 'indicados'),
 ('y', 'expensas', 'aquí', 'indicados'),
 ('Bajas', 'expensas', 'sin', 'encargado'),
 ('las', 'expensas', 'aquí', 'indicadas'),
 ('y', 'expensas', 'están', 'sujetos'),
 ('y', 'expensas', 'consignadas', 'en'),
 ('Bajas', 'expensas', 'y', 'ABL'),
 ('de', 'expensas', 'y', 'ABL'),
 ('Bajas', 'expensas', '4000', 'aproximadamente'),
 ('de', 'expensas', 'mensualesAntigüedad', 'estimada'),
 ('las', 'expensas', 'que', 'pueden'),
 ('y', 'expensas', 'consignadas', 'son'),
 ('y', 'expensas', 'aquí', 'indicados'),
 ('bajas', 'expensas', 'y', 'precio'),
 ('y', 'expensas', 'consignadas', 'en'),
 ('y', 'expensas', 'aquí', 'indicados'),
 ('y', 'expensas', 'aquí', 'indicados'),
 ('bajas', 'expensas', 'Cuenta', 'con'),
 ('las', 'expensas', 'puede', 'variar'),
 ('de', 'expensas', 'y', 'ABL'),
 ('en', 'expensas', 'más', 'bajas'),
 ('y', 'expensas', 'indicados', 'están'),
 ('de', 'expensas', 'responden', 'a'),
 ('de', 'expensas', 'y', 'ABL'),
 ('de', 'expensas', 'res

Al observar los valores de las expensas podemos determinar que algunos de gran interés son: bajas, bajísimas, mínimas, altas, medias, intermedias, accesibles, sin (no tiene expensas), bajísimas.

Podemos determinar los valores que tomarán los aspectos en las nuevas columnas del dataset mirando la frecuencia y la importancia de cada uno de ellos:

Se hace un  análisis a ojo de las palabras sucesores y antecesoras más frecuentes halladas usando Regex.

Si bien el método de  Minqing Hu y Bing Liu se usa para hallar valores que indiquen algún tipo de sentimiento para algún posible aspecto, se realizará una pequeña modificación del mismo, de forma de tomar en cuenta valores que aporten información interesante a cada posible aspecto y no solo aquellas que indiquen sentimientos.

1-Balcón: Se observan con gran frecuencia las siguientes palabras, "con", "Con", "Un", "un". A su vez se observaron palabras interesantes como "hermoso", "amplio" o, "tiene".

Las palabras más frecuentes dan a entender que la propiedad tiene balcón. Por otra parte el grupo de palabras marcadas como interesantes resulta de interés ya que no solo mencionan la existencia de un balcón, sino que describen sus características e indican algún tipo de sentimiento. 

Las palabras "hermoso", "amplio", "tiene",  "con", "Con", "Un", "un"  serán candidatas a los posibles valores que el aspecto balcón puede tomar.

2-Av: Solo resultan interesantes aquellas palabras que indican el nombre de alguna avenida de la ciudad (CABA, Argentina). Sin embargo se descarta tomando en cuenta la dificultad de nombrar a todas las avenidas y todas las posibles formas en que son escritas en una descripción.

3-Frente: La palabra "al" presenta una frecuencia muy alta entre todas las palabras sucesores antecesoras y sucesoras. Se observan palabras interesantes como "Hospital", "Urquiza", "terminal", "con balcón". 

Estas ultimas son de frecuencia muy baja, a pesar de que resultan interesantes, no aportarían gran información sobre el total de descripciones dada la baja frecuencia de las mismas. La palabra "al" tampoco aporta información relevante para la búsqueda de descripciones interesantes.

Por lo tanto, frente es descartada como aspecto a ser incluido como columna en el dataset.

4-Pisos: Se observan palabras frecuentes como "con". Sin embargo las palabras que aportan más información son aquellas que hacen referencia a la cantidad de pisos, o al material del que esté hecho del mismo.

Materiales/característica del mismo: Mármol, madera, cerámica, parquet, flotantes, pinotea, cemento, porcelanato, ladrillo.

Resulta de interés la cantidad de piso y el material del que está hecho.

5-Expensas: En lugar de buscar palabras frecuentes, únicamente se hizo una observación respecto a palabras antecesoras y sucesoras. Tales son, "bajas", "tiene", "incluye", "pesos", "ABL", "bajísimas", "sin".

En base a los valores más frecuentes para los aspectos observados anteriormente se realiza una lista de valores frecuentes para cada aspecto. 

In [81]:
def cantidad_de_pisos():
    return list(range(1, 12)) #tomamos como piso maximo 12

In [42]:
def agregar_cantidad_pisos(lista_a_llenar):
    
    lista_de_pisos = cantidad_de_pisos()

    for i in lista_de_pisos:
        lista_a_llenar.append(str(i))


In [43]:
aspectos_balcon = ["HERMOSO", "Hermoso", "hermoso", "AMPLIO", "Amplio", "amplio", "tiene", "TIENE", "Tiene"]

aspectos_expensas = ["BAJAS", "bajas", "Bajas", "BAJISIMAS", "BAJÍSIMAS", "bajisimas", "bajísimas", "Bajisimas", "Bajísimas", "INLCUYE", "incluye", "Incluye", "ABL", "abl", "TIENE", "tiene", "SIN", "sin", "Sin"]

#Aspectos pisos se parte en dos, posibles valores para el tipo de material, y posibles valores
#para la cantidad de pisos.

aspectos_material_pisos = ["MARMOL", "marmol", "Marmol", "MÁRMOL", "mármol", "Mármol","MADERA", "madera", "Madera", "CERÁMICA", "cerámica", "Cerámica", "CERAMICA", "ceramica", "Ceramica", "PARQUET", "parquet", "Parquet", "FLOTANTES", "flotantes", "Flotantes", "PINOTEA", "pinotea","Pinotea", "CEMENTO", "cemento", "Cemento", "PORCELANTATO", "porcelanato", "Porcelanato", "LADRILLO", "ladrillo", "Ladrillo"]

aspectos_cantidad_pisos = []
agregar_cantidad_pisos(aspectos_cantidad_pisos)


Las listas de palabras anteriormente creadas son de utilidad en la búsqueda de valores para cada uno de los aspectos puestos como nuevas columnas en el dataset. 

A continuación se crea una función que verifica si en una descripción se encuentra algunos de los posibles valores que puede tomar un aspecto.

In [44]:
#primer parametro: string, segundo parametro: lista de strings.
def contiene_valor(descripcion_a_analizar, lista_de_valores_posibles):

    for valor in lista_de_valores_posibles:

        if valor in descripcion_a_analizar:
            return valor
    
    return False

In [45]:
#testeo de la funcion contiene_valor
palabra = contiene_valor("Balcon hermoso", aspectos_balcon)
print(palabra)
palabra = contiene_valor("balcon amplio", aspectos_balcon)
print(palabra)

hermoso
amplio


Ahora se crea una nueva función que inserte los valores de los aspectos de cada descripción en una lista 

In [46]:
def insertar_valores_de_aspectos(lista_descripciones,lista_valores,lista_valores_posibles):

    for descripcion in lista_descripciones:
        
        valor = contiene_valor(descripcion, lista_valores_posibles)

        if(valor != False):
            lista_valores.append(valor)
        else:
            lista_valores.append("NaN")
        

Se insertan en listas los valores de los aspectos (balcón, piso, expensas) para cada descripción. Esto se aplica para el conjunto de train y test.

In [47]:
#ds_train
ds_properati_train
balcon_train = []
expensas_train = []
material_pisos_train = []
cantidad_pisos_train = []


insertar_valores_de_aspectos(ds_properati_train["descripcion"], balcon_train, aspectos_balcon)
insertar_valores_de_aspectos(ds_properati_train["descripcion"], expensas_train, aspectos_expensas)
insertar_valores_de_aspectos(ds_properati_train["descripcion"], material_pisos_train, aspectos_material_pisos)
insertar_valores_de_aspectos(ds_properati_train["descripcion"], cantidad_pisos_train, aspectos_cantidad_pisos)

In [48]:
print(balcon_train)
print(expensas_train)
print(material_pisos_train)
print(cantidad_pisos_train)

abl', 'abl', 'abl', 'sin', 'NaN', 'abl', 'abl', 'ABL', 'NaN', 'abl', 'Bajas', 'NaN', 'NaN', 'abl', 'abl', 'abl', 'abl', 'abl', 'NaN', 'Incluye', 'abl', 'NaN', 'BAJAS', 'NaN', 'NaN', 'ABL', 'NaN', 'abl', 'abl', 'ABL', 'abl', 'abl', 'abl', 'abl', 'NaN', 'abl', 'abl', 'NaN', 'incluye', 'abl', 'NaN', 'ABL', 'NaN', 'abl', 'abl', 'tiene', 'bajas', 'NaN', 'abl', 'abl', 'sin', 'NaN', 'abl', 'tiene', 'abl', 'abl', 'abl', 'abl', 'NaN', 'NaN', 'NaN', 'abl', 'NaN', 'NaN', 'bajas', 'Bajas', 'abl', 'abl', 'Bajas', 'NaN', 'abl', 'abl', 'NaN', 'NaN', 'abl', 'abl', 'ABL', 'NaN', 'NaN', 'bajas', 'abl', 'NaN', 'abl', 'NaN', 'incluye', 'abl', 'abl', 'bajas', 'abl', 'incluye', 'tiene', 'NaN', 'NaN', 'abl', 'SIN', 'abl', 'bajisimas', 'abl', 'NaN', 'NaN', 'abl', 'Bajas', 'ABL', 'abl', 'ABL', 'NaN', 'abl', 'sin', 'incluye', 'abl', 'incluye', 'NaN', 'sin', 'abl', 'abl', 'abl', 'ABL', 'abl', 'incluye', 'Bajas', 'abl', 'NaN', 'abl', 'Bajas', 'abl', 'bajas', 'abl', 'ABL', 'sin', 'NaN', 'incluye', 'abl', 'TIENE', 

In [49]:
#ds_test
ds_properati_test
balcon_test = []
expensas_test = []
material_pisos_test = []
cantidad_pisos_test = []

insertar_valores_de_aspectos(ds_properati_test["descripcion"], balcon_test, aspectos_balcon)
insertar_valores_de_aspectos(ds_properati_test["descripcion"], expensas_test, aspectos_expensas)
insertar_valores_de_aspectos(ds_properati_test["descripcion"], material_pisos_test, aspectos_material_pisos)
insertar_valores_de_aspectos(ds_properati_test["descripcion"], cantidad_pisos_test, aspectos_cantidad_pisos)

In [50]:
print(balcon_test)
print(expensas_test)
print(material_pisos_test)
print(cantidad_pisos_test)

['amplio', 'NaN', 'NaN', 'amplio', 'amplio', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'Amplio', 'NaN', 'NaN', 'amplio', 'NaN', 'tiene', 'Hermoso', 'amplio', 'amplio', 'amplio', 'amplio', 'hermoso', 'amplio', 'amplio', 'tiene', 'hermoso', 'NaN', 'tiene', 'HERMOSO', 'amplio', 'NaN', 'Hermoso', 'NaN', 'amplio', 'Amplio', 'NaN', 'tiene', 'NaN', 'NaN', 'NaN', 'NaN', 'Amplio', 'amplio', 'Amplio', 'Amplio', 'NaN', 'NaN', 'amplio', 'NaN', 'amplio', 'tiene', 'NaN', 'NaN', 'AMPLIO', 'tiene', 'TIENE', 'NaN', 'Amplio', 'NaN', 'Amplio', 'NaN', 'NaN', 'amplio', 'amplio', 'hermoso', 'NaN', 'NaN', 'NaN', 'amplio', 'amplio', 'NaN', 'amplio', 'amplio', 'amplio', 'hermoso', 'NaN', 'NaN', 'hermoso', 'Hermoso', 'NaN', 'NaN', 'NaN', 'tiene', 'amplio', 'NaN', 'NaN', 'Amplio', 'NaN', 'amplio', 'AMPLIO', 'hermoso', 'amplio', 'tiene', 'NaN', 'NaN', 'NaN', 'Amplio', 'amplio', 'amplio', 'amplio', 'AMPLIO', 'NaN', 'amplio', 'amplio', 'amplio', 'NaN', 'amplio', 'NaN', 'NaN', 'Hermoso', 'NaN', 'NaN', 'Hermoso', 'Na

Vemos que hay palabras que significan lo mismo pero estan escritas distintas, como "marmol" vs "mármol" o "HERMOSO" vs "hermoso". Con el objetivo de que se consideren como una misma clase a la hora de entrenar un modelo, vamos a normalizar estos valores.


In [51]:
#saca tildes y pone las palabras en lower case
def sacar_tildes_string(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s.lower()

In [52]:
def sacar_tildes(lista):
    for i in range(len(lista)):
        lista[i] = sacar_tildes_string(lista[i])

In [53]:
sacar_tildes(balcon_test)
sacar_tildes(balcon_train)
sacar_tildes(expensas_test)
sacar_tildes(expensas_train)
sacar_tildes(material_pisos_test)
sacar_tildes(material_pisos_train)
sacar_tildes(cantidad_pisos_test)
sacar_tildes(cantidad_pisos_train)

Primero se crean nuevas columnas en los ds de train y test que contengan los valores para cada aspecto encontrados anteriormente. 

In [54]:
ds_properati_train = ds_properati_train.assign(balcon = balcon_train, expensas = expensas_train, cantidad_pisos = cantidad_pisos_train, material_pisos = material_pisos_train)

In [55]:
ds_properati_test = ds_properati_test.assign(balcon = balcon_test, expensas = expensas_test, cantidad_pisos = cantidad_pisos_test, material_pisos = material_pisos_test)

In [56]:
ds_properati_train.head()

,id,created_on,latitud,longitud,provincia,barrio,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,descripcion,balcon,expensas,cantidad_pisos,material_pisos
0,Tl/0P3OQvAJx+s3S6k1YTQ==,2021-03-10,-34.611523,-58.410394,Capital Federal,BALVANERA,Venta,Departamento,1.0,1.0,49.0,45.0,79000.0,USD,EN CONSTRUCCION a entregar en Septiembre 202...,nan,nan,1,porcelanato
1,OwVESiGxXI0GQgEX5TuguA==,2021-04-16,-34.621782,-58.424452,Capital Federal,ALMAGRO,Venta,Departamento,1.0,1.0,44.0,40.0,99000.0,USD,Corredor Responsable: Emilia Silvia Pepe - CUC...,amplio,abl,1,marmol
2,LikTulyE2kKBjdKCrPTvOg==,2021-08-28,-34.592425,-58.515042,Capital Federal,VILLA DEVOTO,Venta,PH,3.0,2.0,76.0,66.0,160000.0,USD,Corredor Responsable: Mario Gabriel Gerez - CM...,nan,abl,1,nan
3,yRW3du6JKpSpTVstJJC6Yg==,2021-11-29,-34.607836,-58.446965,Capital Federal,CABALLITO,Venta,Departamento,2.0,1.0,56.0,50.0,153000.0,USD,Departamento en duplex de 56 m2. Cochera fija....,amplio,incluye,1,nan
4,nd95bu4KbYrpHdurPqv3/A==,2021-12-17,-34.561456,-58.467606,Capital Federal,BELGRANO,Venta,Departamento,2.0,1.0,55.0,45.0,159000.0,USD,Estratégicamente ubicado en la uni...,nan,nan,1,nan


In [57]:
ds_properati_test.head()

,id,created_on,latitud,longitud,provincia,barrio,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,descripcion,balcon,expensas,cantidad_pisos,material_pisos
0,Dabq6WG4OfnpGJafX4AH4w==,2021-11-09,-34.552834,-58.491372,Capital Federal,SAAVEDRA,Venta,PH,5.0,3.0,158.000000,115.0,220000.0,USD,"Se vende en el tradicional barrio de ""SAAVEDRA...",amplio,nan,1,nan
1,+b4ao+uuJ47NJGpJMuLP2w==,2021-03-23,-34.580101,-58.412473,Capital Federal,PALERMO,Venta,PH,2.0,1.0,35.000000,32.0,90000.0,USD,Excelente PH en la mejor zona de Palermo.<br><...,nan,abl,1,nan
2,sjZQWQmA3Jgw7HtWWU0bpw==,2021-10-05,-34.598510,-58.411857,Capital Federal,ABASTO,Venta,Departamento,2.0,1.0,43.000000,41.0,85000.0,USD,Corredor Responsable: JUAN FRANCISCO CARATTINO...,nan,abl,1,nan
3,/uazrulgE/zCLdo6x1Xvhw==,2021-05-15,-34.567515,-58.458584,Capital Federal,BELGRANO,Venta,Departamento,2.0,1.0,111.325111,44.0,144000.0,USD,VENTA DEPARTAMENTO<br><br>Impecable departamen...,amplio,abl,1,parquet
4,wirKqwH+3UYdAFVO5NVp4A==,2021-09-02,-34.588866,-58.416343,Capital Federal,PALERMO,Venta,Departamento,2.0,1.0,39.000000,36.0,105000.0,USD,"Departamento de 2 ambientes al frente, apto p...",amplio,nan,2,nan


In [58]:
display(ds_properati_test.cantidad_pisos.unique())
display(ds_properati_test.material_pisos.unique())
display(ds_properati_test.expensas.unique())
display(ds_properati_test.balcon.unique())

array(['1', '2', 'nan', '3', '7', '5', '6', '4', '8', '9'], dtype=object)

array(['nan', 'parquet', 'marmol', 'porcelanato', 'madera', 'ladrillo',
       'ceramica', 'pinotea', 'flotantes', 'cemento'], dtype=object)

array(['nan', 'abl', 'bajas', 'sin', 'incluye', 'tiene', 'bajisimas'],
      dtype=object)

array(['amplio', 'nan', 'tiene', 'hermoso'], dtype=object)

In [59]:
display(ds_properati_train.cantidad_pisos.unique())
display(ds_properati_train.material_pisos.unique())
display(ds_properati_train.expensas.unique())
display(ds_properati_train.balcon.unique())

array(['1', 'nan', '2', '3', '4', '5', '6', '8', '9', '7'], dtype=object)

array(['porcelanato', 'marmol', 'nan', 'madera', 'flotantes', 'parquet',
       'pinotea', 'ceramica', 'ladrillo', 'cemento'], dtype=object)

array(['nan', 'abl', 'incluye', 'bajas', 'sin', 'tiene', 'bajisimas'],
      dtype=object)

array(['nan', 'amplio', 'tiene', 'hermoso'], dtype=object)

Vamos a reeplazar 'tiene' por 'incluye' para unificar estos valores.

In [60]:
ds_properati_test['balcon'] = ds_properati_test['balcon'].replace(['tiene'], 'incluye')
ds_properati_train['balcon'] = ds_properati_train['balcon'].replace(['tiene'], 'incluye')

In [61]:
display(ds_properati_test.cantidad_pisos.unique())
display(ds_properati_test.material_pisos.unique())
display(ds_properati_test.expensas.unique())
display(ds_properati_test.balcon.unique())

array(['1', '2', 'nan', '3', '7', '5', '6', '4', '8', '9'], dtype=object)

array(['nan', 'parquet', 'marmol', 'porcelanato', 'madera', 'ladrillo',
       'ceramica', 'pinotea', 'flotantes', 'cemento'], dtype=object)

array(['nan', 'abl', 'bajas', 'sin', 'incluye', 'tiene', 'bajisimas'],
      dtype=object)

array(['amplio', 'nan', 'incluye', 'hermoso'], dtype=object)

In [62]:
display(ds_properati_train.cantidad_pisos.unique())
display(ds_properati_train.material_pisos.unique())
display(ds_properati_train.expensas.unique())
display(ds_properati_train.balcon.unique())

array(['1', 'nan', '2', '3', '4', '5', '6', '8', '9', '7'], dtype=object)

array(['porcelanato', 'marmol', 'nan', 'madera', 'flotantes', 'parquet',
       'pinotea', 'ceramica', 'ladrillo', 'cemento'], dtype=object)

array(['nan', 'abl', 'incluye', 'bajas', 'sin', 'tiene', 'bajisimas'],
      dtype=object)

array(['nan', 'amplio', 'incluye', 'hermoso'], dtype=object)

In [63]:
#exportamos los datasets 
ds_properati_test.to_csv("datasets/ds_properati_test_ampliado.csv")
ds_properati_train.to_csv("datasets/ds_properati_train_ampliado.csv")

## XGBoost

Entrenaremos un modelo XGBoost para regresión con el nuevo dataset ampliado

En primer lugar será con los hiperparámetros encontrados en el TP1

Luego buscaremos los hiperparámetros optimizados con el nuevo dataset ampliado

XGBoost es una forma más regularizada de Gradient Boosting . XGBoost utiliza la regularización avanzada (L1 y L2), que mejora las capacidades de generalización del modelo.

Ofrece un alto rendimiento en comparación con Gradient Boosting. Su entrenamiento es muy rápido y se puede paralelizar entre clústeres.

Fue diseñado para Big Data, es decir para conjuntos de datos grandes y complejos.

XG Boost puede usarse para problemas de regresión o de clasificación. Pero en este caso lo usaremos para un problema de regresión.

1. Hacer una predicción inicial. Esta predicción puede ser cualquier valor.
2. Construir un árbol para los residuos. Este árbol es diferente a los usados por Gradient Boost. Primero se crea un nodo hoja y se ponen allí **todos** los residuos.
3. Calcular el ***Similarity Score***, para los residuos: $Similarity Score=\frac{(\sum{residuos)^2}}{\#residuos + \lambda}$
4. Tenemos que ver cuál será el siguiente nodo. Para ello vamos a calcular la **ganancia total** (Gain), según escojamos una opción u otra para partir el árbol (qué umbral tomemos). Nos quedamos con el umbral que de la mayor información ganada.
5. Repetimos el anterior hasta alcanzar la profundidad del árbol estipulada.
6. Poda:
    - Elegimos un número al azar → este número se llama gamma ( **𝜸** )
    - Calculamos la diferencia entre el **Gain** del nodo más bajo y **gamma**
    - Si la diferencia es < 0 → removemos el nodo
    - Sino, el nodo se queda y se terminó la poda
7. Volvemos a calcular el árbol (repite paso 3), solo que esta vez usamos lambda 𝛌 igual a 1 al calcular el Similarity Score y el Gain.
8. Podamos (paso quinto nuevamente).
9. Con estos nuevos residuos, construimos un nuevo árbol.
Repetimos todo, desde el paso 2.
Con el nuevo árbol, calculamos la salida de cada elemento y luego los residuos.
Construimos otro árbol.
Seguimos hasta que los residuos son prácticamente cero o bien alcanzamos el número máximo de árboles predefinido.

 Hiper-parámetros

learning_rate: tasa de aprendizaje

max_depth: máxima profundidad de cada árbol

subsample: porcentaje de muestras usadas para cada árbol (valor muy bajo, posible underfitting)

colsample_bytree: porcentaje de features usadas para cada árbol (valores muy alto, posible overfitting)

n_estimators: cantidad de árboles a construir.

objective: función de error a utilizar (algunas: reg:linear para regresión, reg:logistic, binary:logistic para clasificación)

Parámetros de regularización:

gamma: umbral para hacer split basado en la reducción de error de hacer el nuevo split.

alpha: regularización para los pesos de las hojas. Un valor más alto genera una mayor regularización.

lambda: similar alpha pero para la sintonia fina.

Los parametros se agregan al modelo para evitar el overfitting

Armamos el conjunto de train y test

In [64]:
x_train = ds_properati_train.drop(['id','property_price','created_on','provincia','operation','property_currency','descripcion'], axis='columns', inplace=False)
y_train = ds_properati_train.property_price.copy()

x_test = ds_properati_test.drop(['id','property_price','created_on','provincia','operation','property_currency','descripcion'], axis='columns', inplace=False)
y_test = ds_properati_test.property_price.copy()

In [65]:
x_train.head()

,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,balcon,expensas,cantidad_pisos,material_pisos
0,-34.611523,-58.410394,BALVANERA,Departamento,1.0,1.0,49.0,45.0,nan,nan,1,porcelanato
1,-34.621782,-58.424452,ALMAGRO,Departamento,1.0,1.0,44.0,40.0,amplio,abl,1,marmol
2,-34.592425,-58.515042,VILLA DEVOTO,PH,3.0,2.0,76.0,66.0,nan,abl,1,nan
3,-34.607836,-58.446965,CABALLITO,Departamento,2.0,1.0,56.0,50.0,amplio,incluye,1,nan
4,-34.561456,-58.467606,BELGRANO,Departamento,2.0,1.0,55.0,45.0,nan,nan,1,nan


In [66]:
x_test.head()

,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,balcon,expensas,cantidad_pisos,material_pisos
0,-34.552834,-58.491372,SAAVEDRA,PH,5.0,3.0,158.000000,115.0,amplio,nan,1,nan
1,-34.580101,-58.412473,PALERMO,PH,2.0,1.0,35.000000,32.0,nan,abl,1,nan
2,-34.598510,-58.411857,ABASTO,Departamento,2.0,1.0,43.000000,41.0,nan,abl,1,nan
3,-34.567515,-58.458584,BELGRANO,Departamento,2.0,1.0,111.325111,44.0,amplio,abl,1,parquet
4,-34.588866,-58.416343,PALERMO,Departamento,2.0,1.0,39.000000,36.0,amplio,nan,2,nan


Creamos el modelo y lo entrenamos con los hiperparámetros encontrados en el tp1

In [67]:
from sklearn.preprocessing import LabelEncoder

def label_encode(df, campo):
    le = LabelEncoder()
    le.fit(df[campo].unique())
    return le.transform(df[campo])

In [68]:
x_train['barrio'] = label_encode(x_train, 'barrio')
x_test['barrio'] = label_encode(x_test, 'barrio')
x_train['property_type'] = label_encode(x_train, 'property_type')
x_test['property_type'] = label_encode(x_test, 'property_type')
x_train['balcon'] = label_encode(x_train, 'balcon')
x_test['balcon'] = label_encode(x_test, 'balcon')
x_train['expensas'] = label_encode(x_train, 'expensas')
x_test['expensas'] = label_encode(x_test, 'expensas')
x_train['material_pisos'] = label_encode(x_train, 'material_pisos')
x_test['material_pisos'] = label_encode(x_test, 'material_pisos')

x_train['cantidad_pisos'] = x_train['cantidad_pisos'].astype('float')
x_test['cantidad_pisos'] = x_test['cantidad_pisos'].astype('float')

display(x_train.head())
display(x_test.head())

,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,balcon,expensas,cantidad_pisos,material_pisos
0,-34.611523,-58.410394,3,1,1.0,1.0,49.0,45.0,3,4,1.0,9
1,-34.621782,-58.424452,2,1,1.0,1.0,44.0,40.0,0,0,1.0,5
2,-34.592425,-58.515042,47,2,3.0,2.0,76.0,66.0,3,0,1.0,6
3,-34.607836,-58.446965,9,1,2.0,1.0,56.0,50.0,0,3,1.0,6
4,-34.561456,-58.467606,6,1,2.0,1.0,55.0,45.0,3,4,1.0,6


,latitud,longitud,barrio,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,balcon,expensas,cantidad_pisos,material_pisos
0,-34.552834,-58.491372,37,2,5.0,3.0,158.000000,115.0,0,4,1.0,6
1,-34.580101,-58.412473,26,2,2.0,1.0,35.000000,32.0,3,0,1.0,6
2,-34.598510,-58.411857,0,1,2.0,1.0,43.000000,41.0,3,0,1.0,6
3,-34.567515,-58.458584,6,1,2.0,1.0,111.325111,44.0,0,0,1.0,7
4,-34.588866,-58.416343,26,1,2.0,1.0,39.000000,36.0,0,4,2.0,6


In [69]:
!pip install xgboost==1.7.1

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [70]:
import xgboost as xgb

In [80]:
xgb_model = xgb.XGBRegressor(subsample=0.6, objective='reg:tweedie', n_estimators=50, 
                                min_child_weight=5, max_depth=2, gamma=0.3,
                                eval_metric='mae', eta= 0.3, colsample_bytree=0.6, booster='gbtree', seed=0)

xgb_model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.6,
             early_stopping_rounds=None, enable_categorical=False, eta=0.3,
             eval_metric='mae', feature_types=None, gamma=0.3, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=2, max_leaves=0, min_child_weight=5, missing=nan,
             monotone_constraints='()', n_estimators=50, n_jobs=0,
             num_parallel_tree=1, objective='reg:tweedie', ...)

Evaluamos sobre el conjunto de entrenamiento

In [72]:
y_pred_train = xgb_model.predict(x_train)

Evaluamos sobre el conjunto de test para observar su predicción

In [73]:
y_pred_test = xgb_model.predict(x_test)

Calculamos el error para cada predicción:

In [74]:
from sklearn.metrics import mean_squared_error

score_train = xgb_model.score(x_train, y_train)  
print("Score de training: ", score_train)
score_test = xgb_model.score(x_test, y_test)
print("Score de test: ", score_test)

mse_train = mean_squared_error(y_train, y_pred_train)
print("MSE train: %.2f" % mse_train)
print("RMSE train: %.2f" % (mse_train**(1/2.0)))

mse_test = mean_squared_error(y_test, y_pred_test)
print("MSE test: %.2f" % mse_test)
print("RMSE test: %.2f" % (mse_test**(1/2.0)))

Score de training:  0.8024655257603903
Score de test:  0.7833949834199841
MSE train: 1489000224.39
RMSE train: 38587.57
MSE test: 1567552461.77
RMSE test: 39592.33


Como se puede observar, con los mismos hiperparámetros que se usaron para el TP1, el rendimiento de este modelo tuvo cierta mejoría, mejorando el score de training de 0.783 a 0.801 y en testing de 0.677 a 0.780. Además, se puede observar que la diferencia de rendimiento entre training y testing se redujo considerablemente, lo cual indica una reducción del overfitting. Por último, los errores en todos los casos se ven reducidos en cierta medida.

Búsqueda de hiperparámetros

Ahora optimizamos los hiperparametros a través de KFold cross validation con 5 folds con la metrica Mean Absolute Error

In [75]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
xgb.set_config(verbosity=0)
params = {
    'n_estimators':[50, 20],
    'min_child_weight':[4,5], 
    'gamma':[i/10.0 for i in range(3,4)],  
    'subsample':[i/10.0 for i in range(6,8)],
    'colsample_bytree':[i/10.0 for i in range(6,8)], 
    'max_depth': [2,4,7],
    'objective': ['reg:squarederror', 'reg:tweedie'],
    'booster': ['gbtree', 'gblinear'],
    'eval_metric': [mean_absolute_error],
    'eta': [i/10.0 for i in range(3,4)],
}

reg = xgb.XGBRegressor(nthread=-1, random_state=2)

# run randomized search

random_search = RandomizedSearchCV(reg, param_distributions=params,
                                    cv=5, scoring='neg_mean_squared_error', random_state=2)

random_search.fit(x_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                   param_distributions={'booster': ['gbtree', 'gblinear'],
                                        'colsample_bytree': [0.6, 0.7],
                                        'eta': [0.3],
                                        'eval_metric': [<function mean_absolute_error at 0x7fa1975a8ca0>],
                                        'gamma': [0.3], 'max_depth': [2, 4, 7],
                                        'min_child_weight': [4, 5],
                                        'n_estimators': [50, 20],
                                        'objective': ['reg:squarederror',
                                                      'reg:tweedie'],
                                        'subsample': [0.6, 0.7]},
                   random_state=2, scoring='neg_mean_squared_error')

Obtenemos el mejor estimador (mejores parámetros)

In [76]:
display(random_search.best_params_)
xgbmodel_cv = random_search.best_estimator_

{'subsample': 0.7,
 'objective': 'reg:squarederror',
 'n_estimators': 50,
 'min_child_weight': 5,
 'max_depth': 7,
 'gamma': 0.3,
 'eval_metric': <function sklearn.metrics._regression.mean_absolute_error(y_true, y_pred, *, sample_weight=None, multioutput='uniform_average')>,
 'eta': 0.3,
 'colsample_bytree': 0.7,
 'booster': 'gbtree'}

Ahora hacemos predicciones con los conjuntos y evaluamos la performance

In [77]:
y_pred_train_cv = xgbmodel_cv.predict(x_train)
y_pred_tes_cvt = xgbmodel_cv.predict(x_test)

Observamos las metricas con el conjunto de train y test

In [78]:
score_train = xgbmodel_cv.score(x_train, y_train)  
print("Score de training: ", score_train)
score_test = xgbmodel_cv.score(x_test, y_test)
print("Score de test: ", score_test)

mse_train = mean_squared_error(y_train, y_pred_train_cv)
print("MSE train: %.2f" % mse_train)
print("RMSE train: %.2f" % (mse_train**(1/2.0)))

mse_test = mean_squared_error(y_test, y_pred_tes_cvt)
print("MSE test: %.2f" % mse_test)
print("RMSE test: %.2f" % (mse_test**(1/2.0)))

Score de training:  0.9083923771523523
Score de test:  0.8452222378252134
MSE train: 690531470.52
RMSE train: 26277.97
MSE test: 1120113771.86
RMSE test: 33468.10


Comparando exclusivamente con el modelo entrenado con el dataset ampliado, se puede ver una gran mejoría en el rendimiento de training y, en una medida menor, para training. Si bien esto puede ser un indicio de overfitting, el score alcanzado en training es altamente satisfactorio. Adicionalmente, se puede volver a ver una gran mejoría en el cálculo de los errores.

In [79]:
# Exportamos el modelo
xgbmodel_cv.save_model('models/xgb_ampliado.json')

/root/venv/lib/python3.9/site-packages/xgboost/sklearn.py:787: UserWarning: eval_metric is not saved in Scikit-Learn meta.
  warnings.warn(


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8953c3ab-dbb1-46a7-ac7e-fe8a0929b2fc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>